<a href="https://colab.research.google.com/github/shubham-bari/Fed-Learning-From-Scratch/blob/main/FederatedLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import random as rand
import math

In [2]:
df = pd.read_csv(r'/content/data_banknote_authentication.csv')
df.head(), df.shape

(   Feature A  Feature B  Feature C  Feature D  Output
 0    3.62160     8.6661    -2.8073   -0.44699       0
 1    4.54590     8.1674    -2.4586   -1.46210       0
 2    3.86600    -2.6383     1.9242    0.10645       0
 3    3.45660     9.5228    -4.0112   -3.59440       0
 4    0.32924    -4.4552     4.5718   -0.98880       0,
 (1372, 5))

In [3]:
m,n = df.shape

In [4]:
randseq = np.random.permutation(m)
df_1 = df.iloc[randseq[0:m], :]
df_1.shape

(1372, 5)

In [5]:
df_1.head()

,Feature A,Feature B,Feature C,Feature D,Output
195,-1.8584,7.8860,-1.6643,-1.83840,0
404,0.3798,0.7098,0.7572,-0.44440,0
487,3.4776,8.8110,-3.1886,-0.92285,0
514,2.0153,1.8479,3.1375,0.42843,0
737,4.1542,7.2756,-2.4766,-1.20990,0


In [6]:
trainval = math.floor(m*0.9)
train_data = df_1.iloc[:trainval, :]
val_data = df_1.iloc[trainval: , :]

train_data.shape, val_data.shape

((1234, 5), (138, 5))

In [7]:
N = 5   #Number of clients

#we need to split training data into 10 equal parts
#every part will get trainval/N data points

parts = trainval//N
remainder = trainval%N
split_data=[]

for i in range(N):
  split_data.append(train_data.iloc[i*parts:(i+1)*parts, :])


In [8]:
for i in range(remainder):
    leftover_row = train_data.iloc[[parts*N + i]]
    split_data[i] = pd.concat([split_data[i], leftover_row], ignore_index=True)

In [9]:
sum=0
for i in range(N):
  print(split_data[i].shape)
  sum+=split_data[i].shape[0]
print(sum)

(247, 5)
(247, 5)
(247, 5)
(247, 5)
(246, 5)
1234


In [10]:
#Now the data is Split perfectly

In [11]:
val_data.shape

(138, 5)

In [12]:
X_test = val_data.iloc[:, :n-1].T
Y_test = val_data.iloc[:, n-1]

X_test.shape, Y_test.shape

((4, 138), (138,))

In [13]:
np.array(Y_test).reshape(1, m-trainval).shape

(1, 138)

In [49]:
import warnings
warnings.filterwarnings('ignore')
def sigmoid(x):
  sig = 1 / (1 + np.exp(-x.astype(np.float128)))     # Define sigmoid function
  sig = np.minimum(sig, 0.9999)  # Set upper bound
  sig = np.maximum(sig, 0.0001)  # Set lower bound
  return sig


In [36]:
#Initializing the global model

In [65]:
Layers = 4
Lx = [n-1, 10, 10, 1]
np.random.seed(3)

W1_g=np.random.randn(Lx[1], Lx[0])
b1_g=np.random.randn(Lx[1], 1)

W2_g=np.random.randn(Lx[2], Lx[1])
b2_g=np.random.randn(Lx[2], 1)

W3_g=np.random.randn(Lx[3], Lx[2])
b3_g=np.random.randn(Lx[3], 1)

pd.DataFrame(W1_g).to_csv(r'/content/W1_avg.csv', header=False, index=False)
pd.DataFrame(b1_g).to_csv(r'/content/b1_avg.csv', header=False, index=False)

pd.DataFrame(W2_g).to_csv(r'/content/W2_avg.csv', header=False, index=False)
pd.DataFrame(b2_g).to_csv(r'/content/b2_avg.csv', header=False, index=False)

pd.DataFrame(W3_g).to_csv(r'/content/W3_avg.csv', header=False, index=False)
pd.DataFrame(b3_g).to_csv(r'/content/b3_avg.csv', header=False, index=False)

In [66]:
lr = 0.05
iterations=500
rounds=15

loss = np.zeros((rounds,1))
acc = np.zeros((rounds,1))

In [67]:
for r in range(rounds):

  for c in range(N):

    m, n = split_data[c].shape
    X_train = split_data[c].iloc[:, :n-1].T
    Y_train = np.array(split_data[c].iloc[:, n-1]).reshape(1, m)

    W1 = W1_g
    b1 = b1_g
    W2 = W2_g
    b2 = b2_g
    W3 = W3_g
    b3 = b3_g

    for i in range(iterations):

      #Forward propagation
      Z1 = W1_g.dot(X_train)+b1      #(10, 1097)
      A1 = sigmoid(Z1)          #(10, 1097)

      Z2 = W2_g.dot(A1)+b2     #(10, 1097)
      A2 = sigmoid(Z2)          #(10, 1097)

      #W3(1, 10)
      Z3 = W3_g.dot(A2)+b3     #(1, 1097)
      A3 = sigmoid(Z3)

      eps = 1e-8
      pred = A3>0.5

      dA3 = A3*(1-A3)         #(1, 1097)
      dZ3 = A3 - np.array(Y_train).reshape(1,-1)   #dZ3(1, 1097)
      dW3 = 1/m * np.dot(dZ3, A2.T)        #A2(10,1097)    #W3(1,10)
      db3 = 1/m * np.sum(dZ3, axis=1, keepdims=True)


      dA2 = A2*(1-A2)    #(10, 1097)
      dZ2 = np.dot(W3.T, dZ3)*dA2        #dZ2(10, 1097)
      dW2 = 1/m * np.dot(dZ2, A1.T)      #A1(10, 1097)  #W2(10, 10)
      db2 = 1/m * np.sum(dZ2, axis=1, keepdims=True)

      dA1 = A1*(1-A1)                #(10, 1097)
      dZ1 = np.dot(W2.T, dZ2)*dA1    #dZ1(10, 1097)
      dW1 = 1/m * np.dot(dZ1, X_train.T)
      db1 = 1/m * np.sum(dZ1, axis=1, keepdims=True)


      W1 = W1 - lr*dW1
      b1 = b1 - lr*db1

      W2 = W2 - lr*dW2
      b2 = b2 - lr*db2

      W3 = W3 - lr*dW3
      b3 = b3 - lr*db3

    W1_g = W1_g + W1
    b1_g = b1_g + b1
    W2_g = W2_g + W2
    b2_g = b2_g + b2
    W3_g = W3_g + W3
    b3_g = b3_g + b3

  pd.DataFrame(W1_g).to_csv(f'/content/W1_avg_r{r+1}.csv', header=False, index=False)
  pd.DataFrame(b1).to_csv(f'/content/b1_avg_r{r+1}.csv', header=False, index=False)

  pd.DataFrame(W2_g).to_csv(f'/content/W2_avg_r{r+1}.csv', header=False, index=False)
  pd.DataFrame(b2_g).to_csv(f'/content/b2_avg_r{r+1}.csv', header=False, index=False)

  pd.DataFrame(W3_g).to_csv(f'/content/W3_avg_r{r+1}.csv', header=False, index=False)
  pd.DataFrame(b3_g).to_csv(f'/content/b3_avg_r{r+1}.csv', header=False, index=False)

  W1_g = W1_g/N
  b1_g = b1_g/N
  W2_g = W2_g/N
  b2_g = b2_g/N
  W3_g = W3_g/N
  b3_g = b3_g/N

  Z1 = W1_g.dot(X_train)+b1      #(10, 1097)
  A1 = sigmoid(Z1)          #(10, 1097)

  Z2 = W2_g.dot(A1)+b2     #(10, 1097)
  A2 = sigmoid(Z2)          #(10, 1097)

  #W3(1, 10)
  Z3 = W3_g.dot(A2)+b3     #(1, 1097)
  A3 = sigmoid(Z3)

  pred = A3>0.5
  acc[r] = np.sum(abs(pred==Y_train))/m
  loss[r]=-np.sum(Y_train*np.log(A3+eps) + (1-Y_train)*np.log(1-A3+eps))/m

  print(f"For round {r+1}: Accuracy: {(acc[r])}, Loss: {(loss[r])}")





For round 1: Accuracy: [0.7195122], Loss: [1.19529268]
For round 2: Accuracy: [0.81300813], Loss: [1.72568759]
For round 3: Accuracy: [0.80081301], Loss: [1.83464015]
For round 4: Accuracy: [0.80081301], Loss: [1.83464015]
For round 5: Accuracy: [0.80081301], Loss: [1.83464015]
For round 6: Accuracy: [0.79268293], Loss: [1.90951934]
For round 7: Accuracy: [0.7804878], Loss: [2.02183813]
For round 8: Accuracy: [0.70325203], Loss: [2.73319043]
For round 9: Accuracy: [0.96747967], Loss: [0.29961676]
For round 10: Accuracy: [0.63821138], Loss: [3.33222395]
For round 11: Accuracy: [0.54065041], Loss: [4.23077424]
For round 12: Accuracy: [0.72357724], Loss: [2.54599246]
For round 13: Accuracy: [0.84552846], Loss: [1.42280461]
For round 14: Accuracy: [0.92682927], Loss: [0.67401271]
For round 15: Accuracy: [0.63821138], Loss: [3.33222395]
